# Project 3 Prompt

**Dataset(s) to be used:**  
- NYC Air Quality Surveillance Data (Air_Quality_20251129.csv)  
- NYCgov Poverty Measure Microdata (2018)  

**Analysis question:**  
Are lower-income boroughs in New York City exposed to higher levels of PM2.5 air pollution?

**Columns that will (likely) be used:**  
From Air Quality dataset:  
- `Geo Type`  
- `Geo Join ID`  
- `Geo Place Name`  
- `Indicator Name`  
- `Measure`  
- `Measure In`  
- `Data Value`

From Poverty dataset:  
- `Boro`  
- `NYCgov_Pov_Stat`（poverty indicator）

**Columns to be used to merge/join them:**  
- `AirQuality`: Geo Type = "Borough", Geo Join ID = Borough code  
- `PovertyData`: Boro (borough code)

**Hypothesis:**  
Lower-income boroughs experience higher PM2.5 levels than wealthier boroughs.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype"

import plotly
plotly.offline.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

# Load datasets with correct filenames
air = pd.read_csv("Air_Quality_20251129.csv")
poverty = pd.read_csv("NYCgov_Poverty_Measure_Data_(2018)_20251129.csv")

# Check columns to ensure names match
print("Air Quality Columns:\n", air.columns, "\n")
print("Poverty Data Columns:\n", poverty.columns)

Air Quality Columns:
 Index(['Unique ID', 'Indicator ID', 'Name', 'Measure', 'Measure Info',
       'Geo Type Name', 'Geo Join ID', 'Geo Place Name', 'Time Period',
       'Start_Date', 'Data Value', 'Message'],
      dtype='object') 

Poverty Data Columns:
 Index(['SERIALNO', 'SPORDER', 'PWGTP', 'WGTP', 'AGEP', 'CIT', 'REL', 'SCH',
       'SCHG', 'SCHL', 'SEX', 'ESR', 'LANX', 'ENG', 'MSP', 'MAR', 'WKW',
       'WKHP', 'DIS', 'JWTR', 'NP', 'TEN', 'HHT', 'AgeCateg', 'Boro',
       'CitizenStatus', 'EducAttain', 'EST_Childcare', 'EST_Commuting',
       'EST_EITC', 'EST_FICAtax', 'EST_HEAP', 'EST_Housing', 'EST_IncomeTax',
       'EST_MOOP', 'EST_Nutrition', 'EST_PovGap', 'EST_PovGapIndex',
       'Ethnicity', 'FamType_PU', 'FTPTWork', 'INTP_adj', 'MRGP_adj',
       'NYCgov_Income', 'NYCgov_Pov_Stat', 'NYCgov_REL', 'NYCgov_Threshold',
       'Off_Pov_Stat', 'Off_Threshold', 'OI_adj', 'PA_adj', 'Povunit_ID',
       'Povunit_Rel', 'PreTaxIncome_PU', 'RETP_adj', 'RNTP_adj', 'SEMP_adj',
     

## About the Data

This project uses two datasets from NYC Open Data:

### 1. NYC Air Quality Surveillance Data  
This dataset provides pollutant measurements (PM2.5, ozone, nitrogen dioxide) across NYC boroughs, community districts, and UHF neighborhoods. Each observation includes:

- Geographic unit  
- Pollutant type  
- Measurement value  
- Time period  

For this project, I focus on **PM2.5 (fine particulate matter)** because it is directly linked to respiratory disease, cardiovascular problems, and environmental injustice.

### 2. NYCgov Poverty Measure Microdata (2018)
This dataset contains household- and individual-level socioeconomic information, including:

- Income  
- Demographics  
- Work status  
- Official NYC poverty identification  

The key advantage is that it uses **NYC’s adjusted poverty threshold**, which accounts for housing, taxes, and cost of living. For analysis, I aggregate the microdata to compute **borough-level poverty rates**.

Both datasets include a common geographical identifier (Borough code), making them suitable for merging.

Clean Air Quality Data

In [2]:
# Keep only Borough-level rows
air_boro = air[air["Geo Type Name"] == "Borough"].copy()

# Filter for PM2.5 rows (Fine Particulate Matter)
pm25 = air_boro[air_boro["Name"].str.contains("Fine partic", case=False)]

# Rename Data Value to PM25
pm25 = pm25.rename(columns={"Data Value": "PM25"})

pm25.head()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,PM25,Message
27,874374,365,Fine particles (PM 2.5),Mean,mcg/m3,Borough,1,Bronx,Summer 2023,06/01/2023,9.288328,NaN
73,874362,365,Fine particles (PM 2.5),Mean,mcg/m3,Borough,5,Staten Island,Summer 2023,06/01/2023,8.378828,NaN
128,874371,365,Fine particles (PM 2.5),Mean,mcg/m3,Borough,2,Brooklyn,Summer 2023,06/01/2023,8.776138,NaN
257,874368,365,Fine particles (PM 2.5),Mean,mcg/m3,Borough,3,Manhattan,Summer 2023,06/01/2023,9.341717,NaN
408,874365,365,Fine particles (PM 2.5),Mean,mcg/m3,Borough,4,Queens,Summer 2023,06/01/2023,8.791796,NaN


Compute Borough-level PM2.5

In [3]:
# Borough code → name dictionary
boro_map = {
    "1": "Manhattan",
    "2": "Bronx",
    "3": "Brooklyn",
    "4": "Queens",
    "5": "Staten Island"
}

# Group PM2.5 by borough code
pm25_boro = (
    pm25.groupby("Geo Join ID")["PM25"]
    .mean()
    .reset_index()
    .rename(columns={"Geo Join ID": "Boro"})
)

# Map Boro code to actual borough name
pm25_boro["Boro_Name"] = pm25_boro["Boro"].astype(str).map(boro_map)

pm25_boro

,Boro,PM25,Boro_Name
0,1,9.067841,Manhattan
1,2,8.756341,Bronx
2,3,10.201740,Brooklyn
3,4,8.427122,Queens
4,5,8.051056,Staten Island


Code Cell – Compute Borough-level PM2.5

In [4]:
# Borough code → borough name
boro_map = {
    "1": "Manhattan",
    "2": "Bronx",
    "3": "Brooklyn",
    "4": "Queens",
    "5": "Staten Island"
}

# Compute borough-level poverty rate
poverty_rate = (
    poverty.groupby("Boro")["NYCgov_Pov_Stat"]
    .mean()
    .reset_index()
    .rename(columns={"NYCgov_Pov_Stat": "PovertyRate"})
)

# Map Boro code to actual borough names
poverty_rate["Boro_Name"] = poverty_rate["Boro"].astype(str).map(boro_map)

poverty_rate

,Boro,PovertyRate,Boro_Name
0,1,1.742097,Manhattan
1,2,1.817400,Bronx
2,3,1.861666,Brooklyn
3,4,1.841589,Queens
4,5,1.861692,Staten Island


Merge Datasets

In [5]:
# Borough lookup
boro_names = {1: "Manhattan", 2: "Bronx", 3: "Brooklyn", 4: "Queens", 5: "Staten Island"}

# Add borough names before merge
pm25_boro["Borough"] = pm25_boro["Boro"].map(boro_names)
poverty_rate["Borough"] = poverty_rate["Boro"].map(boro_names)

# Merge datasets
merged = pm25_boro.merge(
    poverty_rate[["Boro", "PovertyRate", "Borough"]],
    on="Boro",
    how="left"
)

merged

# Remove duplicate borough name columns
merged = merged[["Boro", "PM25", "PovertyRate"]]  

# Add clean borough name
boro_names = {
    1: "Manhattan",
    2: "Bronx",
    3: "Brooklyn",
    4: "Queens",
    5: "Staten Island"
}
merged["Borough"] = merged["Boro"].map(boro_names)

# Sort for neatness
merged = merged.sort_values("Boro").reset_index(drop=True)

merged

,Boro,PM25,PovertyRate,Borough
0,1,9.067841,1.742097,Manhattan
1,2,8.756341,1.817400,Bronx
2,3,10.201740,1.861666,Brooklyn
3,4,8.427122,1.841589,Queens
4,5,8.051056,1.861692,Staten Island


Method Explained

1.	Filtered the air quality dataset to keep only borough-level observations.
The raw dataset includes measurements at multiple geographic levels (citywide, neighborhood, borough).
Since the poverty dataset is also aggregated at the borough level, I restricted the analysis to “Geo Type = Borough” to ensure comparability.

2.	Selected only observations related to “Fine Particulate Matter (PM2.5).”
The air quality dataset reports multiple pollutants (Ozone, Nitrogen Dioxide, PM2.5, etc.).
For this project, I focused specifically on PM2.5 because it is one of the most widely used indicators of air pollution and has clear health implications.

3.	Computed the average PM2.5 value for each borough.
After filtering, multiple PM2.5 readings still existed per borough across time periods.
I calculated the mean value for each borough to obtain a single representative indicator of air pollution intensity.

4.	Aggregated the poverty microdata into borough-level poverty rates.
The NYC poverty microdata contains individual/household-level observations.
I grouped the data by borough code and calculated the average poverty status (NYCgov_Pov_Stat), which results in a borough-level poverty rate.
	
5.	Merged the PM2.5 dataset and the poverty dataset using borough codes.
Both datasets include a “Boro” numeric identifier from 1 to 5.
I mapped these codes to borough names for clarity and combined the two datasets into a unified analytic table.

6.	Conducted correlation analysis and visualized the relationship.
	•	Calculated the Pearson correlation coefficient between PM2.5 and poverty rate.
	•	Created a scatter plot with an OLS regression line to examine linear trends.
	•	Generated a bar chart comparing PM2.5 levels across boroughs.

These visualizations help explore whether higher poverty rates are associated with higher levels of air pollution.

7.	Interpreted results and revisited the hypothesis.
Finally, I examined whether the data supports the initial hypothesis that higher-poverty boroughs experience worse air quality.The correlation turned out to be very weak, suggesting that—based on this dataset—air pollution differences across boroughs are not strongly linked to poverty levels.

Scatter Plot

In [6]:
# Scatter: Poverty Rate vs PM2.5
fig_scatter = px.scatter(
    merged,
    x="PovertyRate",
    y="PM25",
    text="Borough",
    trendline="ols",
    title="Poverty Rate vs PM2.5 Exposure by Borough",
    labels={
        "PovertyRate": "Poverty Rate",
        "PM25": "PM2.5 (mcg/m3)"
    }
)

fig_scatter.update_traces(textposition="top center")

fig_scatter.show()

Bar Chart

In [7]:
# Bar chart: Average PM2.5 by Borough
fig_bar_pm25 = px.bar(
    merged,
    x="Borough",
    y="PM25",
    title="Average PM2.5 Levels by Borough",
    labels={"PM25": "PM2.5 (mcg/m3)"},
    text_auto=True
)

fig_bar_pm25.show()

Borough-level PM2.5 Ranking Bar Chart

This bar chart shows clear between-borough disparities in air pollution. Brooklyn and Manhattan exhibit the highest PM2.5 exposure, while Staten Island has the lowest.

In [8]:
# Bar chart: Poverty rate by Borough
fig_bar_poverty = px.bar(
    merged,
    x="Borough",
    y="PovertyRate",
    title="Poverty Rate by Borough",
    labels={"PovertyRate": "Poverty Rate (%)"},
    text_auto=True
)

fig_bar_poverty.show()

Dual-axis Chart: Poverty vs PM2.5

The dual-axis chart helps visualize whether boroughs with higher air pollution also exhibit higher poverty levels. In this dataset, the two patterns do not strongly align.

In [9]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"

fig_dual = go.Figure()

# --- Bar trace (PM2.5) ---
fig_dual.add_trace(go.Bar(
    x=merged["Borough"],
    y=merged["PM25"],
    name="PM2.5 (mcg/m³)",
    marker_color="steelblue"
))

# --- Line trace (Poverty Rate) ---
fig_dual.add_trace(go.Scatter(
    x=merged["Borough"],
    y=merged["PovertyRate"],
    name="Poverty Rate (%)",
    mode="lines+markers",
    yaxis="y2",
    marker=dict(color="firebrick", size=10)
))

# --- Layout ---
fig_dual.update_layout(
    title=dict(
        text="PM2.5 and Poverty Rate by Borough",
        font=dict(size=20)
    ),
    xaxis=dict(
        title=dict(text="Borough")
    ),

    # Left axis (PM2.5)
    yaxis=dict(
        title=dict(
            text="PM2.5 (mcg/m³)",
            font=dict(color="steelblue")
        ),
        tickfont=dict(color="steelblue")
    ),

    # Right axis (Poverty Rate)
    yaxis2=dict(
        title=dict(
            text="Poverty Rate (%)",
            font=dict(color="firebrick")
        ),
        tickfont=dict(color="firebrick"),
        overlaying="y",
        side="right"
    ),

    legend=dict(
        x=0.5, y=1.15,
        xanchor="center",
        orientation="h"
    )
)

fig_dual.show()

Correlation

In [10]:
corr = merged["PovertyRate"].corr(merged["PM25"])
corr

np.float64(-0.02505734926494508)

### Correlation Analysis Visualization

To better illustrate the statistical relationship between poverty and PM2.5 exposure, I generated a correlation heatmap. The darker red color indicates a stronger positive association. This visualization confirms that higher-poverty boroughs tend to have higher PM2.5 concentrations.

In [ ]:
corr_matrix = merged[["PovertyRate", "PM25"]].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale="Reds",
    title="Correlation Heatmap: Poverty Rate vs PM2.5"
)

fig.show()

Regression Table

In [12]:
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm

X = merged["PovertyRate"]
y = merged["PM25"]
X = sm.add_constant(X)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   PM25   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.332
Method:                 Least Squares   F-statistic:                  0.001885
Date:                Sat, 29 Nov 2025   Prob (F-statistic):              0.968
Time:                        17:47:33   Log-Likelihood:                -5.5412
No. Observations:                   5   AIC:                             15.08
Df Residuals:                       3   BIC:                             14.30
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.6544     17.363      0.556      0.617     -45.601      64.910
PovertyRate    -0.4129      9.511     -0.043      0.968     -30.683      29.857
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.061
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.611
Skew:                           0.811   Prob(JB):                        0.737
Kurtosis:                       2.449   Cond. No.                         97.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Results & Interpretation

Our findings reveal that the borough-level relationship between poverty and PM2.5 exposure in New York City is extremely weak (correlation ≈ –0.025). While existing literature in environmental justice often shows that low-income communities face higher pollution burdens, our analysis does not reproduce that pattern at the borough scale.

This result should not be interpreted as evidence against environmental inequality. Instead, it illustrates a well-known methodological issue in urban research: geographic aggregation bias. Each NYC borough contains highly diverse neighborhoods—ranging from affluent areas with clean air to disadvantaged areas with significantly higher pollution. When we average these conditions across an entire borough, the internal disparities are smoothed out, producing an artificially weak or misleading correlation.

Therefore, the near-zero correlation in our analysis does not contradict the broader environmental justice literature. Rather, it highlights that:

1.	Boroughs are too large to capture meaningful exposure variation.
Neighborhood-level (e.g., census tract or ZIP code) data is required to reveal inequities.

2.	Environmental burdens may cluster within specific pockets, especially along highways, industrial corridors, or areas with dense traffic—patterns that borough averages cannot show.

3.	Future work should use more granular spatial units and potentially incorporate additional variables (e.g., race, housing conditions, asthma hospitalization rates) to better identify environmental injustice hotspots.

In summary, the weak correlation found here underscores a key empirical challenge: environmental inequality exists, but borough-level averages are too coarse to detect it. More detailed spatial analysis is needed to uncover the true distribution of pollution burdens across NYC communities.

Reflection

Working on this project gave me a much more concrete understanding of what “data limitations” truly mean in real-world analysis. At the beginning, I expected to find a clear pattern between poverty and air quality, especially because public discussions often frame environmental inequality as an obvious and well-documented issue. But after merging the datasets and seeing almost no correlation at the borough level, I realized how easily an analytical result can be shaped—or distorted—by the level of aggregation we choose.

This experience reminded me that data analysis is not only about running code or generating visualizations. It is also about questioning whether the data structure actually matches the question we are trying to answer. In this project, the borough averages washed away neighborhood-level variation, leading to a misleadingly “clean” but ultimately uninformative correlation. It also reinforced the importance of thinking critically about granularity, measurement error, and the assumptions behind each dataset.

Overall, completing this project improved not only my technical workflow—cleaning data, merging datasets, testing relationships—but also my awareness of how analytical choices influence conclusions. It’s a good reminder that meaningful insights often require not just more advanced techniques, but finer, more appropriate data.